Version 1.1.0

# Mean encodings

In this programming assignment you will be working with `1C` dataset from the final competition. You are asked to encode `item_id` in 4 different ways:

    1) Via KFold scheme;  
    2) Via Leave-one-out scheme;
    3) Via smoothing scheme;
    4) Via expanding mean scheme.

**You will need to submit** the correlation coefficient between resulting encoding and target variable up to 4 decimal places.

### General tips

* Fill NANs in the encoding with `0.3343`.
* Some encoding schemes depend on sorting order, so in order to avoid confusion, please use the following code snippet to construct the data frame. This snippet also implements mean encoding without regularization.

In [1]:
import pandas as pd
import numpy as np
from itertools import product
from grader import Grader
import matplotlib.pyplot as plt
import math

# Read data

In [2]:
sales = pd.read_csv('../readonly/final_project_data/sales_train.csv.gz')

In [3]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [4]:
len(sales)

2935849

# Aggregate data

Since the competition task is to make a monthly prediction, we need to aggregate the data to montly level before doing any encodings. The following code-cell serves just that purpose.

In [5]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})

#fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

C:\Users\pfsch\AppData\Roaming\Python\Python37\site-packages\pandas\core\groupby\generic.py:1315: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


In [6]:
all_data.head()

,shop_id,item_id,date_block_num,target
139255,0,19,0,0.0
141495,0,27,0,0.0
144968,0,28,0,0.0
142661,0,29,0,0.0
138947,0,32,0,6.0


In [7]:
global_mean = 0.3343

# Mean encodings without regularization

After we did the techinical work, we are ready to actually *mean encode* the desired `item_id` variable. 

Here are two ways to implement mean encoding features *without* any regularization. You can use this code as a starting point to implement regularized techniques. 

#### Method 1

In [8]:
# Calculate a mapping: {item_id: target_mean}
group_target_by_item_id = all_data.groupby('item_id').target

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_id_target_mean_1'] = all_data['item_id'].map(group_target_by_item_id.mean())

# Fill NaNs
all_data['item_id_target_mean_1'].fillna(global_mean, inplace=True) 

# Print correlation
encoded_feature = all_data['item_id_target_mean_1'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621764


In [9]:
all_data.head()

,shop_id,item_id,date_block_num,target,item_id_target_mean_1
139255,0,19,0,0.0,0.022222
141495,0,27,0,0.0,0.056834
144968,0,28,0,0.0,0.141176
142661,0,29,0,0.0,0.037383
138947,0,32,0,6.0,1.319042


#### Method 2

In [10]:
'''
     Differently to `.target.mean()` function `transform` 
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_id_target_mean_2'] = group_target_by_item_id.transform('mean')

# Fill NaNs
all_data['item_id_target_mean_2'].fillna(global_mean, inplace=True) 

# Print correlation
encoded_feature = all_data['item_id_target_mean_2'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.4830386988621764


In [11]:
all_data.head()

,shop_id,item_id,date_block_num,target,item_id_target_mean_1,item_id_target_mean_2
139255,0,19,0,0.0,0.022222,0.022222
141495,0,27,0,0.0,0.056834,0.056834
144968,0,28,0,0.0,0.141176,0.141176
142661,0,29,0,0.0,0.037383,0.037383
138947,0,32,0,6.0,1.319042,1.319042


See the printed value? It is the correlation coefficient between the target variable and your new encoded feature. You need to **compute correlation coefficient** between the encodings, that you will implement and **submit those to coursera**.

In [12]:
grader = Grader()

# 1. KFold scheme

Explained starting at 41 sec of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

**Now it's your turn to write the code!** 

You may use 'Regularization' video as a reference for all further tasks.

First, implement KFold scheme with five folds. Use KFold(5) from sklearn.model_selection. 

1. Split your data in 5 folds with `sklearn.model_selection.KFold` with `shuffle=False` argument.
2. Iterate through folds: use all but the current fold to calculate mean target for each level `item_id`, and  fill the current fold.

    *  See the **Method 1** from the example implementation. In particular learn what `map` and pd.Series.map functions do. They are pretty handy in many situations.

In [13]:
all_data.head()

,shop_id,item_id,date_block_num,target,item_id_target_mean_1,item_id_target_mean_2
139255,0,19,0,0.0,0.022222,0.022222
141495,0,27,0,0.0,0.056834,0.056834
144968,0,28,0,0.0,0.141176,0.141176
142661,0,29,0,0.0,0.037383,0.037383
138947,0,32,0,6.0,1.319042,1.319042


In [14]:
from sklearn import model_selection

In [15]:
all_data['k_fold'] = np.nan
kf  = model_selection.KFold(n_splits=5, shuffle=False)

In [16]:
for tr_ind, val_ind in kf.split(all_data):
    X_tr, X_val = all_data.iloc[tr_ind].copy(), all_data.iloc[val_ind].copy()
    means = X_val['item_id'].map(X_tr.groupby('item_id').target.mean())
    X_val['k_fold'] = means
    all_data.iloc[val_ind] = X_val
all_data.head()

,shop_id,item_id,date_block_num,target,item_id_target_mean_1,item_id_target_mean_2,k_fold
139255,0,19,0,0.0,0.022222,0.022222,NaN
141495,0,27,0,0.0,0.056834,0.056834,0.048523
144968,0,28,0,0.0,0.141176,0.141176,0.142424
142661,0,29,0,0.0,0.037383,0.037383,0.030303
138947,0,32,0,6.0,1.319042,1.319042,0.894020


In [17]:
# Fill nans with global mean
all_data['k_fold'].fillna(global_mean,inplace=True)
all_data.head()

,shop_id,item_id,date_block_num,target,item_id_target_mean_1,item_id_target_mean_2,k_fold
139255,0,19,0,0.0,0.022222,0.022222,0.334300
141495,0,27,0,0.0,0.056834,0.056834,0.048523
144968,0,28,0,0.0,0.141176,0.141176,0.142424
142661,0,29,0,0.0,0.037383,0.037383,0.030303
138947,0,32,0,6.0,1.319042,1.319042,0.894020


In [18]:
# YOUR CODE GOES HERE

# You will need to compute correlation like that
corr = np.corrcoef(all_data['target'].values, all_data['k_fold'].values)[0][1]
print(corr)
grader.submit_tag('KFold_scheme', corr)

0.41645907127988446
Current answer for task KFold_scheme is: 0.41645907127988446


# 2. Leave-one-out scheme

Now, implement leave-one-out scheme. Note that if you just simply set the number of folds to the number of samples and run the code from the **KFold scheme**, you will probably wait for a very long time. 

To implement a faster version, note, that to calculate mean target value using all the objects but one *given object*, you can:

1. Calculate sum of the target values using all the objects.
2. Then subtract the target of the *given object* and divide the resulting value by `n_objects - 1`. 

Note that you do not need to perform `1.` for every object. And `2.` can be implemented without any `for` loop.

It is the most convenient to use `.transform` function as in **Method 2**.

In [19]:
# Calculate sum of target values by item_id
by_id_sums = group_target_by_item_id.sum()
print('Sums:', by_id_sums)

by_id_counts = group_target_by_item_id.count()
print('Counts:', by_id_counts)

Sums: item_id
0           1.0
1           6.0
2           2.0
3           2.0
4           1.0
5           1.0
6           1.0
7           1.0
8           2.0
9           1.0
10          1.0
11          1.0
12          1.0
13          1.0
14          1.0
15          1.0
16          1.0
17          1.0
18          1.0
19          1.0
20          1.0
21          1.0
22          1.0
23          1.0
24          1.0
25          1.0
26          1.0
27         42.0
28         84.0
29         12.0
          ...  
22140     287.0
22141      53.0
22142      10.0
22143     865.0
22144     116.0
22145     320.0
22146      29.0
22147      61.0
22148       2.0
22149      35.0
22150      58.0
22151     371.0
22152      85.0
22153       9.0
22154      59.0
22155      74.0
22156       4.0
22157       4.0
22158       1.0
22159      15.0
22160      49.0
22161       1.0
22162     699.0
22163      75.0
22164     493.0
22165       2.0
22166     348.0
22167    1320.0
22168       6.0
22169       1.0
Name: targ

In [20]:
# perform (sum-target)/(n-1)
all_data['leave_one_out'] = np.nan
n_objects = len(all_data)

all_data['leave_one_out'] = ((all_data['item_id'].map(by_id_sums) - 
                            all_data['target']) / 
                            (all_data['item_id'].map(by_id_counts)-1))

all_data.head()

,shop_id,item_id,date_block_num,target,item_id_target_mean_1,item_id_target_mean_2,k_fold,leave_one_out
139255,0,19,0,0.0,0.022222,0.022222,0.334300,0.022727
141495,0,27,0,0.0,0.056834,0.056834,0.048523,0.056911
144968,0,28,0,0.0,0.141176,0.141176,0.142424,0.141414
142661,0,29,0,0.0,0.037383,0.037383,0.030303,0.037500
138947,0,32,0,6.0,1.319042,1.319042,0.894020,1.316088


In [21]:
# YOUR CODE GOES HERE

corr = np.corrcoef(all_data['target'].values, all_data['leave_one_out'].values)[0][1]
print(corr)
grader.submit_tag('Leave-one-out_scheme', corr)

0.4803848311293092
Current answer for task Leave-one-out_scheme is: 0.4803848311293092


# 3. Smoothing

Explained starting at 4:03 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Next, implement smoothing scheme with $\alpha = 100$. Use the formula from the first slide in the video and $0.3343$ as `globalmean`. Note that `nrows` is the number of objects that belong to a certain category (not the number of rows in the dataset).

From my notes:

* If category is large, we can trusted the encoding because we have a lot of samples. But if the category is rare, then we can't.  
* $\alpha$ controls the amount of regularization  
* Only works together with some other regularization method- can comebine with CV loop

Formula:  
$\frac{\bar{x}_T n_{\text{ROWS}} + \bar{x}_G*\alpha}{n_{\text{ROWS}} + \alpha}$

$\bar{x}_T$: Target mean  
$\bar{x}_G$: Global mean  
$n_{\text{ROWS}}$: Number of rows 

In [22]:
alpha = 100;

In [23]:
# make use of previous groupby objects
all_data['smoothing'] = np.nan

all_data['smoothing'] = ((all_data['item_id_target_mean_1']*all_data['item_id'].map(by_id_counts) + global_mean * alpha) / (
                          all_data['item_id'].map(by_id_counts)+alpha))
all_data.head()

,shop_id,item_id,date_block_num,target,item_id_target_mean_1,item_id_target_mean_2,k_fold,leave_one_out,smoothing
139255,0,19,0,0.0,0.022222,0.022222,0.334300,0.022727,0.237448
141495,0,27,0,0.0,0.056834,0.056834,0.048523,0.056911,0.089905
144968,0,28,0,0.0,0.141176,0.141176,0.142424,0.141414,0.168964
142661,0,29,0,0.0,0.037383,0.037383,0.030303,0.037500,0.107910
138947,0,32,0,6.0,1.319042,1.319042,0.894020,1.316088,1.260635


In [24]:
# YOUR CODE GOES HERE

corr = np.corrcoef(all_data['target'].values, all_data['smoothing'].values)[0][1]
print(corr)
grader.submit_tag('Smoothing_scheme', corr)

0.48181987970972573
Current answer for task Smoothing_scheme is: 0.48181987970972573


# 4. Expanding mean scheme

Explained starting at 5:50 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Finally, implement the *expanding mean* scheme. It is basically already implemented for you in the video, but you can challenge yourself and try to implement it yourself. You will need [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) and [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html) functions from pandas.

In [25]:
cumsum = group_target_by_item_id.cumsum() - all_data['target']
cumcnt = all_data.groupby('item_id').cumcount()

In [26]:
all_data['expanding_mean'] = cumsum/cumcnt
all_data['expanding_mean'].fillna(global_mean,inplace=True)
all_data.head()

,shop_id,item_id,date_block_num,target,item_id_target_mean_1,item_id_target_mean_2,k_fold,leave_one_out,smoothing,expanding_mean
139255,0,19,0,0.0,0.022222,0.022222,0.334300,0.022727,0.237448,0.3343
141495,0,27,0,0.0,0.056834,0.056834,0.048523,0.056911,0.089905,0.3343
144968,0,28,0,0.0,0.141176,0.141176,0.142424,0.141414,0.168964,0.3343
142661,0,29,0,0.0,0.037383,0.037383,0.030303,0.037500,0.107910,0.3343
138947,0,32,0,6.0,1.319042,1.319042,0.894020,1.316088,1.260635,0.3343


In [27]:
all_data.tail()

,shop_id,item_id,date_block_num,target,item_id_target_mean_1,item_id_target_mean_2,k_fold,leave_one_out,smoothing,expanding_mean
10768834,59,22162,33,0.0,1.556793,1.556793,0.020000,1.560268,1.334117,1.560268
10769024,59,22163,33,0.0,0.581395,0.581395,0.334300,0.585938,0.473493,0.585938
10769690,59,22164,33,0.0,1.235589,1.235589,0.334300,1.238693,1.054970,1.238693
10771216,59,22166,33,0.0,0.295918,0.295918,0.349227,0.296170,0.298926,0.296170
10770511,59,22167,33,0.0,1.081081,1.081081,1.231425,1.081967,1.024550,1.081967


In [28]:
# YOUR CODE GOES HERE

corr = np.corrcoef(all_data['target'].values, all_data['expanding_mean'])[0][1]
print(corr)
grader.submit_tag('Expanding_mean_scheme', corr)

0.5025245211081716
Current answer for task Expanding_mean_scheme is: 0.5025245211081716


## Authorization & Submission
To submit assignment parts to Cousera platform, please, enter your e-mail and token into variables below. You can generate token on this programming assignment page. Note: Token expires 30 minutes after generation.

In [29]:
STUDENT_EMAIL = 'pfschus@umich.edu'
STUDENT_TOKEN = 
grader.status()

You want to submit these numbers:
Task KFold_scheme: 0.41645907127988446
Task Leave-one-out_scheme: 0.4803848311293092
Task Smoothing_scheme: 0.48181987970972573
Task Expanding_mean_scheme: 0.5025245211081716


In [ ]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)